I removed some of the graph and featueres calculated in FraudFeatureEngineer I found that those features were the problem. They were leaking information future information into the present.  For example: Given everything up to December, here's what October's fraud behavior looks like. So leakage was coming from the way we are calculating features. 


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
import community as community_louvain
import networkx as nx

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import xgboost as xgb
from xgboost import XGBClassifier


In [2]:
df = pd.read_csv('/Users/noimotbakare/Dropbox/Fraud_Payments/data/fraud_payment_data', sep=',', header=0)


In [3]:
#Converting time into a Datetime object
df['timestamp'] = pd.to_datetime(df['Time_step'])
#Removing redudant columns
df=df.drop('Time_step',axis=1)
df=df.drop('Sender_lob',axis=1)

In [4]:
#Engineering Features that do not rely on Statistics before train test split

#First Digit 
df['USD_amount'] = pd.to_numeric(df['USD_amount'], errors='coerce')
#Extract the first digit from each amount
df['first_digit'] = df['USD_amount'].astype(str).str.strip().str[0]
df = df[df['first_digit'].str.isdigit()]
df['first_digit'] = df['first_digit'].astype(int)

#Last Digit 
#Extract the last digit after decimal USD_amount
df['last_digit_after_dec'] = df['USD_amount'].astype(str).str.split('.').str[1].str[-1]
df = df[df['last_digit_after_dec'].str.isdigit()]
df['last_digit_after_dec'] = df['last_digit_after_dec'].astype(int)

In [5]:


df = df.sort_values('timestamp') #( I know the data may be sorted, but I like to cross my t's and dot my i's.)
split_date = df['timestamp'].quantile(0.80)

train_full = df[df['timestamp'] <= split_date].copy()
#future set test set 
df_test = df[df['timestamp'] > split_date].copy()

#stratifying to address class imbalance 
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(
    train_full,
    #20% of train_full becomes validation
    test_size=0.20,  
    #stratifying on fraud 
    stratify=train_full['Label'], #I know we said the data has done a good job splititng but again ^
    random_state=42
)

Sorting and Stratifying although the data is sorted for best practice. 

In [6]:

df = df.sort_values('timestamp') #( I know the data may be sorted, but I like to cross my t's and dot my i's.)
split_date = df['timestamp'].quantile(0.80)

train_full = df[df['timestamp'] <= split_date].copy()
#future set test set 
df_test = df[df['timestamp'] > split_date].copy()

#stratifying to address class imbalance 
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(
    train_full,
    #20% of train_full becomes validation
    test_size=0.20,  
    #stratifying on fraud 
    stratify=train_full['Label'], #I know we said the data has done a good job splititng but again ^
    random_state=42
)

In [7]:
# Make sure that the distribution of fraudulent/legitimate transactions are consistent across the three different sets.
print('The distribution of fraud for the train data is:\n', df_train['Label'].value_counts(normalize=True))
print('The distribution of fraud for the validation set is:\n', df_val['Label'].value_counts(normalize=True))
print('The distribution of fraud for the test set is:\n', df_test['Label'].value_counts(normalize=True))

The distribution of fraud for the train data is:
 Label
0    0.979287
1    0.020713
Name: proportion, dtype: float64
The distribution of fraud for the validation set is:
 Label
0    0.979287
1    0.020713
Name: proportion, dtype: float64
The distribution of fraud for the test set is:
 Label
0    0.979996
1    0.020004
Name: proportion, dtype: float64


1. Rolling sender/beneficiary behavior √
filled large numbers with 99999
filled NaN with 0 because it means 

here I also calculate 'sender_last_time_diff' - that is the sudden burst - how much time passed since the sender's previous transaction

2. One-hot encode Transaction Type √

3. Lagged target encoding (fraud rates) √
Lagged target encoding (fraud rates) -  computing historical fraud rates for an entity up to but not including the current transaction.

This answers the quesrion What was the fraud rate for this sender / country / sector before this transaction?

4. Time aware country/sector behavior √
Initially I calculated: 
Country_Fraud_Ratio (lifetime or globally)
Sector_Fraud_Ratio (lifetime) 

Here we look at:
country mean fraud rate -  in this country over last N transactions, BEFORE this one
sector mean fraud rate - in this sector over last N transactions, BEFORE this one

5. Incremental Network Fraud Signals 
(I do not use Louvain or NX - if anyone wants to give it a try - I just want to keep things clean in hopes we get some results). So this asks if and how fast does fraud spread through a network

6. Tomorrow I may try this-  We can look at how a user's network changes overtime


In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from collections import defaultdict

class FraudFeatureEngineer_TimeAware(BaseEstimator, TransformerMixin):

    def __init__(self, windows=[7,30], te_window=200):
        self.windows = windows
        self.te_window = te_window
        self.fitted_ = False

    def fit(self, X, y=None):
        self.fitted_ = True
        return self

    def _create_rolling_features(self, df):
        df = df.sort_values('timestamp').copy()
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.reset_index(drop=True)
        df['ts'] = df['timestamp']

#---------
#Tx features - rolling w= 7, 30, 90 days - how many transactions this sender made in the past w days, before the current one.
#---------
        for w in self.windows:
            window = f"{w}d"

            df[f"sender_tx_count_{w}d"] = (
                df.groupby('Sender_Account')
                  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].count().shift(1))
                  .reset_index(level=0, drop=True)
                  .fillna(0)
            )

            df[f"sender_tx_mean_{w}d"] = (
                df.groupby('Sender_Account')
                  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].mean().shift(1))
                  .reset_index(level=0, drop=True)
                  .fillna(0)
            )

            df[f"sender_tx_std_{w}d"] = (
                df.groupby('Sender_Account')
                  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].std().shift(1))
                  .reset_index(level=0, drop=True)
                  .fillna(0)
            )

            df[f"bene_tx_count_{w}d"] = (
                df.groupby('Bene_Account')
                  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].count().shift(1))
                  .reset_index(level=0, drop=True)
                  .fillna(0)
            )

            df[f"bene_tx_mean_{w}d"] = (
                df.groupby('Bene_Account')
                  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].mean().shift(1))
                  .reset_index(level=0, drop=True)
                  .fillna(0)
            )

            df[f"bene_tx_std_{w}d"] = (
                df.groupby('Bene_Account')
                  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].std().shift(1))
                  .reset_index(level=0, drop=True)
                  .fillna(0)
            )
            
#Sender last time - how much time passed since the sender's previous transaction
#sudden burst
        df['sender_last_time_diff'] = (
            df.groupby('Sender_Account')['ts']
              .diff()
              .dt.total_seconds()
              .shift(1)
              .fillna(999999)
        )

        df = df.drop(columns=['ts'])
        return df

#------------ 
#Lagged features 
#--------------
    def _add_lagged_target_encoding(self, df):
        df = df.sort_values(["timestamp"])

        df["sender_fraud_rate"] = (
            df.groupby("Sender_Account")["Label"]
              .rolling(window=self.te_window, min_periods=1)
              .mean()
              .shift(1)
              .reset_index(level=0, drop=True)
        )

        df["country_fraud_rate"] = (
            df.groupby("Sender_Country")["Label"]
              .rolling(window=self.te_window, min_periods=1)
              .mean()
              .shift(1)
              .reset_index(level=0, drop=True)
        )

        df["sector_fraud_rate"] = (
            df.groupby("Sender_Sector")["Label"]
              .rolling(window=self.te_window, min_periods=1)
              .mean()
              .shift(1)
              .reset_index(level=0, drop=True)
        )

        return df

 #-----------
 # Incremental Graph Fraud Features      
 #--------------------------------
    #Does fraud spread through the network overtime? how fast?
    def _add_incremental_graph_features(self, df):
        df = df.sort_values("timestamp").copy()
         #dynamic neighbors
        neighbors = defaultdict(set)

        #features
        fraud_neighbors_count = []
        fraud_neighbors_rate = []
        connected_to_fraud = []

        #rolling fraud (memory)
        recent_frauds = defaultdict(list)
        window_days = 30

        for idx, row in df.iterrows():
            s = row["Sender_Account"]
            b = row["Bene_Account"]
            t = row["timestamp"]
            #alright so we want the neighbors seen before "this" transasction
            neighbor_set = neighbors[s] | neighbors[b]

            #Valid frauds - will be frauds within said time window 
            valid_frauds = [ts for ts in recent_frauds[s] + recent_frauds[b] if (t - ts).days <= window_days]

            fraud_neighbors_count.append(len(valid_frauds))
            connected_to_fraud.append(1 if len(valid_frauds) > 0 else 0)
            
            #rate = count/total neightbors 
            fraud_neighbors_rate.append(len(valid_frauds) / max(1, len(neighbor_set)))
            #this line update the neighbor graph after the features are recorded
            neighbors[s].add(b)
            neighbors[s].add(b)
            neighbors[b].add(s)

            #if fraud - then record for future use 
            if row["Label"] == 1:
                recent_frauds[s].append(t)
                recent_frauds[b].append(t)

        df["fraud_neighbors_count_30d"] = fraud_neighbors_count
        df["fraud_neighbors_rate_30d"] = fraud_neighbors_rate
        df["connected_to_fraud_30d"] = connected_to_fraud

        return df

    def transform(self, X):
        if not self.fitted_:
            raise RuntimeError("Call fit() first.")

        df = X.copy()

        df = self._create_rolling_features(df)
        df = self._add_lagged_target_encoding(df)
        df = self._add_incremental_graph_features(df)

        df = df.fillna(0)
        return df


This takes a little less than 30minutes to run

In [9]:
fe = FraudFeatureEngineer_TimeAware(windows=[7, 30], te_window=200)

fe.fit(df_train)

df_train_transformed = fe.transform(df_train)
df_val_transformed = fe.transform(df_val)
df_test_transformed = fe.transform(df_test)


/var/folders/4z/15wr09ws2bv_y9y731bmm5540000gn/T/ipykernel_64677/3785888919.py:30: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].count().shift(1))
/var/folders/4z/15wr09ws2bv_y9y731bmm5540000gn/T/ipykernel_64677/3785888919.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.rolling(window, on='ts')['USD_amount'].mean().shift(1))
/var/folder

In [10]:
df_train_transformed = df_train_transformed.fillna(0)
df_val_transformed   = df_val_transformed.fillna(0)
df_test_transformed  = df_test_transformed.fillna(0)

In [11]:
#Target Encoding transaction type

cat_cols = ["Transaction_Type"]
df_train_transformed = pd.get_dummies(df_train_transformed, columns=cat_cols)
df_val_transformed   = pd.get_dummies(df_val_transformed,   columns=cat_cols)
df_test_transformed  = pd.get_dummies(df_test_transformed,  columns=cat_cols)

# align columns (val/test may be missing categories)
df_val_transformed  = df_val_transformed.reindex(columns=df_train_transformed.columns, fill_value=0)
df_test_transformed = df_test_transformed.reindex(columns=df_train_transformed.columns, fill_value=0)

In [12]:
#Frequency encoding sender sector and country
for col in ["Sender_Country", "Sender_Sector"]:
    freq = df_train_transformed[col].value_counts()
    df_train_transformed[f"{col}_freq"] = df_train_transformed[col].map(freq)
    df_val_transformed[f"{col}_freq"]   = df_val_transformed[col].map(freq).fillna(0)
    df_test_transformed[f"{col}_freq"]  = df_test_transformed[col].map(freq).fillna(0)

In [13]:
y_train = df_train_transformed['Label']
y_val   = df_val_transformed['Label']
y_test  = df_test_transformed['Label']

In [ ]:
df_train_transformed = df_train_transformed.drop(columns=['Label'])
df_val_transformed   = df_val_transformed.drop(columns=['Label'])
df_test_transformed  = df_test_transformed.drop(columns=['Label'])

In [15]:
#Getting the list of columns in df 
df_train_transformed.columns

Index(['Transaction_Id', 'Sender_Id', 'Sender_Account', 'Sender_Country',
       'Sender_Sector', 'Bene_Id', 'Bene_Account', 'Bene_Country',
       'USD_amount', 'timestamp', 'first_digit', 'last_digit_after_dec',
       'sender_tx_count_7d', 'sender_tx_mean_7d', 'sender_tx_std_7d',
       'bene_tx_count_7d', 'bene_tx_mean_7d', 'bene_tx_std_7d',
       'sender_tx_count_30d', 'sender_tx_mean_30d', 'sender_tx_std_30d',
       'bene_tx_count_30d', 'bene_tx_mean_30d', 'bene_tx_std_30d',
       'sender_last_time_diff', 'sender_fraud_rate', 'country_fraud_rate',
       'sector_fraud_rate', 'fraud_neighbors_count_30d',
       'fraud_neighbors_rate_30d', 'connected_to_fraud_30d',
       'Transaction_Type_DEPOSIT-CASH', 'Transaction_Type_DEPOSIT-CHECK',
       'Transaction_Type_EXCHANGE', 'Transaction_Type_MAKE-PAYMENT',
       'Transaction_Type_MOVE-FUNDS', 'Transaction_Type_PAY-CHECK',
       'Transaction_Type_QUICK-PAYMENT', 'Transaction_Type_WITHDRAWAL',
       'Sender_Country_freq', 'Sende

X_rolling are non network features, and X_rolling_nx includes networkfeatures. 
NOTE: The network calculation do not involve graph network.These are manual calculations. 

Lets check out the performance of the features before addinf network features.

In [17]:
X_rolling = [
'USD_amount', 
 #Benford
'first_digit',
'last_digit_after_dec', 

#Rolling sender Bene behavior 
'sender_tx_count_7d',
'sender_tx_mean_7d',
'sender_tx_std_7d',
'bene_tx_count_7d',
'bene_tx_mean_7d',
'bene_tx_std_7d',
'sender_tx_count_30d',
'sender_tx_mean_30d',
'sender_tx_std_30d', 
'bene_tx_count_30d', 
'bene_tx_mean_30d',
'bene_tx_std_30d', 
'sender_last_time_diff', 


##One Hot Encoding Transaction type 
'Transaction_Type_DEPOSIT-CASH', 
'Transaction_Type_DEPOSIT-CHECK',
'Transaction_Type_EXCHANGE', 
'Transaction_Type_MAKE-PAYMENT',
'Transaction_Type_MOVE-FUNDS', 
'Transaction_Type_PAY-CHECK',
'Transaction_Type_QUICK-PAYMENT', 
'Transaction_Type_WITHDRAWAL',

#Fraud rate
#What was the fraud rate for this sender / country / sector before this transaction?
'sender_fraud_rate',
'country_fraud_rate', 
'sector_fraud_rate',

#frequency encoding sector and counrty
'Sender_Country_freq',
'Sender_Sector_freq'


]

In [18]:
#Does not include network explanatory features 
X_train = df_train_transformed[X_rolling]
X_val   = df_val_transformed[X_rolling]
X_test  = df_test_transformed[X_rolling]


In [19]:
#Initialize scaler on the training data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

#Apply the same scaling to validation and test sets
X_val= scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [20]:
#Finally!!!!!!!!! Run Logistic Baseline Model 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report


#Instantiate the model
#Using class_weight='balanced' to handle known fraud data imbalanc
log_reg = LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced')

#Train the model on the SCALED training data - because Logistic regression is sensetive to scale ( won't need this for XGBoost)
log_reg.fit(X_train, y_train)

y_val_proba_lr = log_reg.predict_proba(X_val)[:, 1]

lr_auc = roc_auc_score(y_val, y_val_proba_lr)
print(f"Logistic Regression Validation ROC-AUC: {lr_auc:.4f}")

y_val_pred_lr = log_reg.predict(X_val)
print("\nValidation Classification Report:")

print(classification_report(y_val, y_val_pred_lr))

Logistic Regression Validation ROC-AUC: 0.6682

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.69      0.82    234744
           1       0.04      0.53      0.07      4965

    accuracy                           0.69    239709
   macro avg       0.51      0.61      0.44    239709
weighted avg       0.97      0.69      0.80    239709



In [27]:
# Compute scale_pos_weight for imbalance
scale_pos_weight_value = (y_train == 0).sum() / (y_train == 1).sum()

# XGBoost model tuned for fraud imbalance
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5, 
    subsample=0.8, 
    colsample_bytree=0.8, 
    eval_metric='logloss',
    scale_pos_weight = scale_pos_weight_value, 
    random_state=42,
    tree_method='hist'
)

# Train
xgb.fit(X_train, y_train)

# Predict probabilities
y_val_proba_xgb = xgb.predict_proba(X_val)[:,1]

# Baseline 0.50 threshold predictions
y_val_pred_xgb = (y_val_proba_xgb >= 0.50).astype(int)

# Metrics
roc_auc = roc_auc_score(y_val, y_val_proba_xgb)
pr_auc  = average_precision_score(y_val, y_val_proba_xgb)

print(f"XGBoost Validation ROC-AUC: {roc_auc:.4f}")
print(f"XGBoost Validation PR-AUC: {pr_auc:.4f}")

print("\nClassification Report @ 0.50 threshold:")
print(classification_report(y_val, y_val_pred_xgb, digits=4))


XGBoost Validation ROC-AUC: 0.7354
XGBoost Validation PR-AUC: 0.1163

Classification Report @ 0.50 threshold:
              precision    recall  f1-score   support

           0     0.9864    0.8201    0.8956    234744
           1     0.0517    0.4634    0.0930      4965

    accuracy                         0.8127    239709
   macro avg     0.5190    0.6418    0.4943    239709
weighted avg     0.9670    0.8127    0.8790    239709



Now we work with the df with the network features 

#Fraud Neighbors ( network features)
'fraud_neighbors_count_30d',
'fraud_neighbors_rate_30d',
'connected_to_fraud_30d',


In [30]:
#nx here just means network
X_rolling_w_nx = [
'USD_amount', 
 #Benford
'first_digit',
'last_digit_after_dec', 

#Rolling sender Bene behavior 
'sender_tx_count_7d',
'sender_tx_mean_7d',
'sender_tx_std_7d',
'bene_tx_count_7d',
'bene_tx_mean_7d',
'bene_tx_std_7d',
'sender_tx_count_30d',
'sender_tx_mean_30d',
'sender_tx_std_30d', 
'bene_tx_count_30d', 
'bene_tx_mean_30d',
'bene_tx_std_30d', 
'sender_last_time_diff', 



##One Hot Encoding Transaction type 
'Transaction_Type_DEPOSIT-CASH', 
'Transaction_Type_DEPOSIT-CHECK',
'Transaction_Type_EXCHANGE', 
'Transaction_Type_MAKE-PAYMENT',
'Transaction_Type_MOVE-FUNDS', 
'Transaction_Type_PAY-CHECK',
'Transaction_Type_QUICK-PAYMENT', 
'Transaction_Type_WITHDRAWAL',

#Fraud rate
#What was the fraud rate for this sender / country / sector before this transaction?
'sender_fraud_rate',
'country_fraud_rate', 
'sector_fraud_rate',

#Fraud Neighbors ( network features)
'fraud_neighbors_count_30d',
'fraud_neighbors_rate_30d',
'connected_to_fraud_30d',

#frequency encoding sector and counrty
'Sender_Country_freq',
'Sender_Sector_freq'
]

In [31]:
#Network explanatory features 
X_train_nx = df_train_transformed[X_rolling_w_nx]
X_val_nx   = df_val_transformed[X_rolling_w_nx]
X_test_nx  = df_test_transformed[X_rolling_w_nx]

In [32]:
#Initialize scaler on the training data for Network explanatory features 
scaler = StandardScaler()
X_train_nx = scaler.fit_transform(X_train_nx)

#Apply the same scaling to validation and test sets
X_val_nx = scaler.transform(X_val_nx)
X_test_nx = scaler.transform(X_test_nx)

Logistic regression using network triaing and validation dfs   

In [ ]:
 #Logistic Baseline Model for nx
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report


#Instantiate the model
#Using class_weight='balanced' to handle known fraud data imbalanc
log_reg_nx = LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced')

#Train the model on the SCALED training data - because Logistic regression is sensetive to scale ( won't need this for XGBoost)
log_reg_nx.fit(X_train_nx, y_train)

y_val_proba_lr_nx = log_reg_nx.predict_proba(X_val_nx)[:, 1]

lr_auc_nx = roc_auc_score(y_val, y_val_proba_lr)
print(f"Nx:Logistic Regression Validation ROC-AUC: {lr_auc:.4f}")

y_val_pred_lr_nx = log_reg_nx.predict(X_val_nx)
print("\nNx:Validation Classification Report:")

print(classification_report(y_val, y_val_pred_lr_nx))

Nx:Logistic Regression Validation ROC-AUC: 0.6682

Nx:Validation Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.78      0.87    234744
           1       0.05      0.49      0.08      4965

    accuracy                           0.78    239709
   macro avg       0.52      0.64      0.48    239709
weighted avg       0.97      0.78      0.86    239709



XGB using network training and validation dfs

In [34]:
# Compute scale_pos_weight for imbalance
scale_pos_weight_value = (y_train == 0).sum() / (y_train == 1).sum()

# XGBoost model tuned for fraud imbalance
xgb_nx = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5, 
    subsample=0.8, 
    colsample_bytree=0.8, 
    eval_metric='logloss',
    scale_pos_weight = scale_pos_weight_value, 
    random_state=42,
    tree_method='hist'
)

# Train
xgb_nx.fit(X_train_nx, y_train)

# Predict probabilities
y_val_proba_xgb_nx = xgb_nx.predict_proba(X_val_nx)[:,1]

# Baseline 0.50 threshold predictions
y_val_pred_xgb_nx = (y_val_proba_xgb_nx >= 0.50).astype(int)

# Metrics
roc_auc_nx = roc_auc_score(y_val, y_val_proba_xgb_nx)
pr_auc_nx = average_precision_score(y_val, y_val_proba_xgb_nx)

print(f"XGBoost Validation ROC-AUC: {roc_auc_nx:.4f}")
print(f"XGBoost Validation PR-AUC: {pr_auc_nx:.4f}")

print("\nClassification Report @ 0.50 threshold:")
print(classification_report(y_val, y_val_pred_xgb_nx, digits=4))


XGBoost Validation ROC-AUC: 0.7683
XGBoost Validation PR-AUC: 0.1342

Classification Report @ 0.50 threshold:
              precision    recall  f1-score   support

           0     0.9872    0.8778    0.9293    234744
           1     0.0742    0.4632    0.1280      4965

    accuracy                         0.8692    239709
   macro avg     0.5307    0.6705    0.5286    239709
weighted avg     0.9683    0.8692    0.9127    239709



Next steps: 

1. Track how a user’s network changes over time (will take the code longer to run)
2. We can play with the threshold to 
3. Try optimizing 
4. Test how much each feature contributes model prediction. 
5. LightGBM
6. SMOTE 
7. Random forest 
